In [1]:
from core.nuum import NuumActions, rnd_sleep
from service.browser import Browser 
import random

from config import NUUM

# NuumBot

### Запускаем браузер и вручную авторизуемся в Nuum

In [2]:
browser = Browser()
nuum = NuumActions(browser.driver)

### Запуск бота в цикле
Бот будет просматривать ленту рекомендаций и подписок поочередно


In [5]:
def run_bot(nuum, N, skip_ratio, mode='rec'):
    open_page = {'rec': nuum.open_rec, 'subs': nuum.open_subs}
    open_page[mode]()
    liked = 0
    for i in range(N):
        if skip_ratio < random.random():
            result = nuum.like_stiker_subscribe(
                like_ratio = 1, 
                stiker_ratio = 1, 
                subscribe_ratio = .5
            )
            if result:
                liked += 1
                print(f'Liked {liked} from {i+1}', end=' ') 
            rnd_sleep(3)
        nuum.next()

In [ ]:
N = 30
skip_ratio = 0.5

nuum = NuumActions(browser.driver)

while True:
    for mode in ['rec', 'subs']:
        try:
            print(f'Start {mode}')
            nuum.open(NUUM)
            run_bot(nuum, N, skip_ratio, mode=mode)
            print()
            
        except Exception as e:
            print(e)
            rnd_sleep(10)
    

Start rec
Kostyushking:  stiker , like
Liked 1 from 3 GAMER005:  stiker , like
Liked 2 from 6 alla_mont:  stiker , like
Liked 3 from 7 Lemoegor:  stiker , like
Liked 4 from 13 Donghua:  stiker , like and subscribe
Liked 5 from 14 

### Открывает ленту рекомменадций, пролистывает и лайкает
*N* - количество пролистываний

In [ ]:
N = 60
skip_ratio = 0.5

#nuum.open_subs()
nuum.open_rec()

liked = 0
for i in range(N):
    
    if skip_ratio < random.random():
        result = nuum.like_stiker_subscribe(
            like_ratio = 1, 
            stiker_ratio = 1, 
            subscribe_ratio = .5
        )
        if result:
            liked += 1
            print(f'Liked {liked} from {i+1}')
        
        rnd_sleep(3)
    else:
        rnd_sleep()
    nuum.next()

### Открывает ленту подписок, пролистывает и лайкает
*N* - количество пролистываний

In [ ]:
N = 120
skip_ratio = 0.5


nuum.open_subs()
#nuum.open_rec()

liked = 0
for i in range(N):
    
    if skip_ratio < random.random():
        
        result = nuum.like_stiker_subscribe(
            like_ratio = 1, 
            stiker_ratio = 1, 
            subscribe_ratio = .5
        )
        if result:
            liked += 1
            print(f'Liked {liked} from {i+1}', end=' ') )
        
        rnd_sleep(3)
    nuum.next()

In [3]:
nuum = NuumActions(browser.driver)
nuum.open(NUUM)